In [2]:
from keras.models import Model
from keras.layers import Input, Dense, Concatenate, Embedding, LSTM
from keras.callbacks import EarlyStopping

from keras.utils import np_utils, plot_model

from sklearn.model_selection import train_test_split

from keras.utils.vis_utils import model_to_dot
from IPython.display import SVG

import numpy as np
import MySQLdb

Using TensorFlow backend.


In [7]:
conn = MySQLdb.connect(
    user='sequel',
    passwd='seQuel9_',
    host='localhost',
    db='coincheck'
)

c = conn.cursor()

In [8]:
query = '''
SELECT
ask.ask64,ask.ask63,ask.ask62,ask.ask61,ask.ask60,ask.ask59,ask.ask58,ask.ask57,ask.ask56,ask.ask55,ask.ask54,ask.ask53,ask.ask52,ask.ask51,ask.ask50,ask.ask49,ask.ask48,ask.ask47,ask.ask46,ask.ask45,ask.ask44,ask.ask43,ask.ask42,ask.ask41,ask.ask40,ask.ask39,ask.ask38,ask.ask37,ask.ask36,ask.ask35,ask.ask34,ask.ask33,ask.ask32,ask.ask31,ask.ask30,ask.ask29,ask.ask28,ask.ask27,ask.ask26,ask.ask25,ask.ask24,ask.ask23,ask.ask22,ask.ask21,ask.ask20,ask.ask19,ask.ask18,ask.ask17,ask.ask16,ask.ask15,ask.ask14,ask.ask13,ask.ask12,ask.ask11,ask.ask10,ask.ask9,ask.ask8,ask.ask7,ask.ask6,ask.ask5,ask.ask4,ask.ask3,ask.ask2,ask.ask1,ask.ask0,
bid.bid0,bid.bid1,bid.bid2,bid.bid3,bid.bid4,bid.bid5,bid.bid6,bid.bid7,bid.bid8,bid.bid9,bid.bid10,bid.bid11,bid.bid12,bid.bid13,bid.bid14,bid.bid15,bid.bid16,bid.bid17,bid.bid18,bid.bid19,bid.bid20,bid.bid21,bid.bid22,bid.bid23,bid.bid24,bid.bid25,bid.bid26,bid.bid27,bid.bid28,bid.bid29,bid.bid30,bid.bid31,bid.bid32,bid.bid33,bid.bid34,bid.bid35,bid.bid36,bid.bid37,bid.bid38,bid.bid39,bid.bid40,bid.bid41,bid.bid42,bid.bid43,bid.bid44,bid.bid45,bid.bid46,bid.bid47,bid.bid48,bid.bid49,bid.bid50,bid.bid51,bid.bid52,bid.bid53,bid.bid54,bid.bid55,bid.bid56,bid.bid57,bid.bid58,bid.bid59,bid.bid60,bid.bid61,bid.bid62,bid.bid63,bid.bid64,
c10.open, c10.high, c10.low, c10.close, c10.amount, c9.open, c9.high, c9.low, c9.close, c9.amount, c8.open, c8.high, c8.low, c8.close, c8.amount, c7.open, c7.high, c7.low, c7.close, c7.amount, c6.open, c6.high, c6.low, c6.close, c6.amount, c5.open, c5.high, c5.low, c5.close, c5.amount, c4.open, c4.high, c4.low, c4.close, c4.amount, c3.open, c3.high, c3.low, c3.close, c3.amount, c2.open, c2.high, c2.low, c2.close, c2.amount, c1.open, c1.high, c1.low, c1.close, c1.amount
FROM array_asks AS ask
	LEFT JOIN array_bids AS bid
	ON ask.datetime = bid.datetime
	LEFT JOIN reg_data AS d
	ON ask.datetime = d.datetime
	LEFT JOIN candle_1m AS c1
	ON d.datetime_round = c1.datetime
	LEFT JOIN candle_1m AS c2
	ON d.datetime_round + 60 = c2.datetime
	LEFT JOIN candle_1m AS c3
	ON d.datetime_round + 120 = c3.datetime
	LEFT JOIN candle_1m AS c4
	ON d.datetime_round + 180 = c4.datetime
	LEFT JOIN candle_1m AS c5
	ON d.datetime_round + 240 = c5.datetime
	LEFT JOIN candle_1m AS c6
	ON d.datetime_round + 300 = c6.datetime
	LEFT JOIN candle_1m AS c7
	ON d.datetime_round + 360 = c7.datetime
	LEFT JOIN candle_1m AS c8
	ON d.datetime_round + 420 = c8.datetime
	LEFT JOIN candle_1m AS c9
	ON d.datetime_round + 480 = c9.datetime
	LEFT JOIN candle_1m AS c10
	ON d.datetime_round + 540 = c10.datetime

WHERE bid.datetime IS NOT NULL
	AND d.later_1m_class IS NOT NULL
	AND c1.open IS NOT NULL
	AND c2.open IS NOT NULL
	AND c3.open IS NOT NULL
	AND c4.open IS NOT NULL
	AND c5.open IS NOT NULL
	AND c6.open IS NOT NULL
	AND c7.open IS NOT NULL
	AND c8.open IS NOT NULL
	AND c9.open IS NOT NULL
	AND c10.open IS NOT NULL

'''
c.execute(query)
x = c.fetchall()

query = '''
SELECT
d.later_1m_class
FROM array_asks AS ask
	LEFT JOIN array_bids AS bid
	ON ask.datetime = bid.datetime
	LEFT JOIN reg_data AS d
	ON ask.datetime = d.datetime
	LEFT JOIN candle_1m AS c1
	ON d.datetime_round = c1.datetime
	LEFT JOIN candle_1m AS c2
	ON d.datetime_round + 60 = c2.datetime
	LEFT JOIN candle_1m AS c3
	ON d.datetime_round + 120 = c3.datetime
	LEFT JOIN candle_1m AS c4
	ON d.datetime_round + 180 = c4.datetime
	LEFT JOIN candle_1m AS c5
	ON d.datetime_round + 240 = c5.datetime
	LEFT JOIN candle_1m AS c6
	ON d.datetime_round + 300 = c6.datetime
	LEFT JOIN candle_1m AS c7
	ON d.datetime_round + 360 = c7.datetime
	LEFT JOIN candle_1m AS c8
	ON d.datetime_round + 420 = c8.datetime
	LEFT JOIN candle_1m AS c9
	ON d.datetime_round + 480 = c9.datetime
	LEFT JOIN candle_1m AS c10
	ON d.datetime_round + 540 = c10.datetime

WHERE bid.datetime IS NOT NULL
	AND d.later_1m_class IS NOT NULL
	AND c1.open IS NOT NULL
	AND c2.open IS NOT NULL
	AND c3.open IS NOT NULL
	AND c4.open IS NOT NULL
	AND c5.open IS NOT NULL
	AND c6.open IS NOT NULL
	AND c7.open IS NOT NULL
	AND c8.open IS NOT NULL
	AND c9.open IS NOT NULL
	AND c10.open IS NOT NULL

'''
c.execute(query)
y = c.fetchall()

In [9]:
X = np.array(x)
Y = np.array(y)
Y = np_utils.to_categorical(Y, 3)
print(X.shape, Y.shape)

(30092, 180) (30092, 3)


In [10]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.8)
print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

(24073, 180) (6019, 180) (24073, 3) (6019, 3)


In [11]:
X_train_order = X_train[:, 0:130]
X_train_candle = X_train[:, 130:].reshape(X_train.shape[0], int((X_train.shape[1]-130)/5), 5)

X_test_order = X_test[:, 0:130]
X_test_candle = X_test[:, 130:].reshape(X_test.shape[0], int((X_test.shape[1]-130)/5), 5)

In [13]:
order_inputs = Input(shape=(130,), name="order_inputs")
candle_inputs = Input(shape=(int((X_train.shape[1]-130)/5), 5), name="candle_inputs")

order_dense = Dense(32, activation='relu')(order_inputs)

candle_dense = LSTM(16)(candle_inputs)

m = concatenate([order_dense, candle_dense], axis=1)
m = Dense(10, activation='relu')(m)

predictions = Dense(3, activation='softmax')(m)

model = Model(inputs=[order_inputs, candle_inputs], outputs=predictions)

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

SVG(model_to_dot(model, show_shapes = True).create(prog='dot', format='svg'))

NameError: name 'concatenate' is not defined

In [112]:
#Early-stopping
# callback = EarlyStopping(monitor='val_acc', patience=4, verbose=1)

# モデルの訓練
history = model.fit([X_train_order, X_train_candle], Y_train,
                    batch_size=1024,
                    epochs=200,
                    verbose=1,
                    validation_split=0.1)

Train on 21665 samples, validate on 2408 samples
Epoch 1/200
21665/21665 [==============================] - 3s - loss: 7.6297 - acc: 0.3157 - val_loss: 5.8394 - val_acc: 0.3248
Epoch 2/200
21665/21665 [==============================] - 1s - loss: 4.1035 - acc: 0.3362 - val_loss: 2.4242 - val_acc: 0.3517
Epoch 3/200
21665/21665 [==============================] - 1s - loss: 2.1447 - acc: 0.3483 - val_loss: 1.6947 - val_acc: 0.3576
Epoch 4/200
21665/21665 [==============================] - 1s - loss: 1.6561 - acc: 0.3556 - val_loss: 1.4903 - val_acc: 0.3463
Epoch 5/200
21665/21665 [==============================] - 1s - loss: 1.4166 - acc: 0.3490 - val_loss: 1.2876 - val_acc: 0.3451
Epoch 6/200
21665/21665 [==============================] - 1s - loss: 1.1615 - acc: 0.3703 - val_loss: 1.1298 - val_acc: 0.3933
Epoch 7/200
21665/21665 [==============================] - 1s - loss: 1.1052 - acc: 0.3940 - val_loss: 1.1101 - val_acc: 0.4045
Epoch 8/200
21665/21665 [==============================

21665/21665 [==============================] - 1s - loss: 1.0199 - acc: 0.4578 - val_loss: 1.1275 - val_acc: 0.3916
Epoch 128/200
21665/21665 [==============================] - 1s - loss: 1.0212 - acc: 0.4585 - val_loss: 1.1331 - val_acc: 0.3870
Epoch 129/200
21665/21665 [==============================] - 1s - loss: 1.0197 - acc: 0.4586 - val_loss: 1.1256 - val_acc: 0.3912
Epoch 130/200
21665/21665 [==============================] - 1s - loss: 1.0197 - acc: 0.4593 - val_loss: 1.1281 - val_acc: 0.3937
Epoch 131/200
21665/21665 [==============================] - 1s - loss: 1.0180 - acc: 0.4602 - val_loss: 1.1316 - val_acc: 0.3920
Epoch 132/200
21665/21665 [==============================] - 1s - loss: 1.0171 - acc: 0.4596 - val_loss: 1.1302 - val_acc: 0.3875
Epoch 133/200
21665/21665 [==============================] - 1s - loss: 1.0169 - acc: 0.4608 - val_loss: 1.1291 - val_acc: 0.3879
Epoch 134/200
21665/21665 [==============================] - 1s - loss: 1.0165 - acc: 0.4613 - val_loss: